<h1> Connect to an ODM2 database and create a variable</h1>

In this notebook we will connect to an ODM2 database, list variable names from which have been loaded from the ODM2 controlled vocabulary, create a new variable and then delete it. 

1) This first block of code sets up the python environment by loading standard python classes as well as IPython display classes and widgets.

In [3]:
import sys
# import code.ui_utils as ui_utils
import os
import getpass
from IPython.display import display, HTML
import ipywidgets as widgets

<p>2) Next load ODM2API database connection class and models </p>
<p>&emsp;   a) each model is mapped to an ODM2 table, ODM2API is available on github and the ODM2API models are here:</p>
<p>&emsp; https://github.com/ODM2/ODM2PythonAPI/blob/master/odm2api/ODM2/models.py </p>

In [4]:
from odm2api.ODMconnection import dbconnection
from odm2api.ODM2.models import *
  

<p>3) Now create text widgets and a container for database connection info.</p>

In [5]:
#print("Enter your ODM2 username") 
container = widgets.Box() 
username_text = widgets.Text(
    value='', placeholder='Enter username',
    description='', disabled=False)
username_output_text = widgets.Text(
    value='', placeholder='Enter username',
    description='Username',disabled=False)
database_address_text = widgets.Text(
    value='', placeholder='Enter database address',
    description='',disabled=False)
database_address_output_text = widgets.Text(
    value='',placeholder='Enter database address',
    description='database address',disabled=False)
database_text = widgets.Text(
    value='', placeholder='Enter database name',
    description='', disabled=False)
database_output_text = widgets.Text(
    value='', placeholder='Enter database name',
    description='database name', disabled=False)
def bind_username_to_output(sender):
    username_output_text.value = username_text.value
def bind_database_address_to_output(sender):
    database_address_output_text.value = database_address_text.value
def bind_database_to_output(sender):
    database_output_text.value = database_text.value     
    
def login(sender):
    #print('Database address : %s, Username: %s, database name: %s' % (
    #    database_address_text.value, username_text.value, database_text.value))
    container.close()    
    
username_text.on_submit(bind_username_to_output)
login_btn = widgets.Button(description="Login")
login_btn.on_click(login)
container.children = [username_text,database_address_text, database_text, login_btn]
container


Box(children=(Text(value=u'', placeholder=u'Enter username'), Text(value=u'', placeholder=u'Enter database address'), Text(value=u'', placeholder=u'Enter database name'), Button(description=u'Login', style=ButtonStyle())))

<p>4) Now enter the password for the user. </p>

In [6]:
print("enter your password: ")
p = getpass.getpass()


enter your password: 
········


5) Now we will use the information entered to establish a database connection. 

In [7]:
#createConnection(self, engine, address, db=None, user=None, password=None, dbtype = 2.0, echo=False)
session_factory = dbconnection.createConnection('postgresql', database_address_text.value, database_text.value, 
                                                username_text.value, p)  
DBSession = session_factory.getSession()




6) We can list available variable names which have been loaded from the ODM2 controlled vocabulary. 
We can query the database based on the model names in ODM2PythonAPI here CVVariableName is here:

https://github.com/ODM2/ODM2PythonAPI/blob/master/odm2api/ODM2/models.py#L142 

In [9]:
retreived_variable = DBSession.query(CVVariableName)
for v in retreived_variable:
    print(v.Name)

Chloroethane
Radon-222
DNA damage, tail length
n-alkane, C31
Methylfluorene
BOD1
Nitrogen, NH3 + NH4
Pheophytin
Chlorophyll b
Asteridae coverage
Chlorophyll fluorescence
Nitrogen, NH4
Zeaxanthin
Intercept
4,4-Methylenebis(2-chloroaniline)
delta-18O of H2O
Water level
Radiation, incoming PAR
Permittivity
Benzo(k)fluoranthene
Methylmercury
1,3-Dimethyladamantane
2,4-Dimethylphenol
Manganese, particulate
2-Methylanthracene
Battery temperature
bis(2-Chloroethyl)ether
Cesium-137
Taxa count
Dinoseb
Permethrin
Bromodichloromethane
Chromium, total
Fluorine
Electric Energy
Fish detections
1-Methylanthracene
9 cis-Neoxanthin
Lead, dissolved
Momentum flux
Nitrogen, dissolved organic
2-Methylnaphthalene
bis-2-chloroisopropyl ether
1-Chloronaphthalene
Iron, particulate
Thorium
Carbon dioxide storage flux
Wind gust speed
BOD3, carbonaceous
Benzo(e)pyrene
Propylene glycol
Odor
Acetate
delta-D of H2O
Acetophenone
1,4,5-Trimethylnaphthalene
Selenium, dissolved
THW Index
Nitrogen, organic kjeldahl
Anili

<h1> 7) Here we create a new ODM2 variable, to do this we fill in the variables fields and save it to the database. </h1><br>
&emsp;&emsp; a) In order to determine how to create a variable we will want to look at the fields that need to be populated <br>
&emsp;&emsp;&emsp; i- Those fields can be found in the variable model here <br> 
&emsp;&emsp;&emsp; ii-https://github.com/ODM2/ODM2PythonAPI/blob/master/odm2api/ODM2/models.py#L392 <br>
&emsp;&emsp; b) The first field, VariableID is the primary key which is automatically populated by the database. You <br>
&emsp;&emsp; shouldn't need to worry about fields ending in ID. <br>
&emsp;&emsp; c) As a result, we first need to specify the variable type. Variable type is part of the ODM2 controlled <br>
&emsp;&emsp; vocabulary. ODM2 Controlled vocabularies can be found here: http://vocabulary.odm2.org/ <br>
&emsp;&emsp; c) The specific controlled vocabulary for variable type is here: http://vocabulary.odm2.org/variabletype/ <br>
&emsp;&emsp; d) Next, we specify the variable code which can be anything we like up to 50 characters. <br>
&emsp;&emsp; e) Now, we specify the the variable name which is also from a controlled vocabulary <br>
&emsp;&emsp; http://vocabulary.odm2.org/variablename/<br>
&emsp;&emsp; f) Next, we specify the variable definition which we can describe however we like, up to 500 characters<br>
&emsp;&emsp; g) Now, we can define the speciation but it can also be left blank this is also a controlled vocabulary. <br>
&emsp;&emsp; http://vocabulary.odm2.org/speciation/ <br>
&emsp;&emsp; h) Finally, we set the NoDataValue field, some analysis may be done for certain specimens but not others, <br>
&emsp;&emsp; in which case it maybe desirable to indicate that by recording a no data value. <br>

8) now print out the variable, it's variable code and save it to the database. 

In [10]:

variable = Variables(VariableTypeCV='Chemistry', VariableCode = 'Tl-particulate', VariableNameCV='Thalium, particulate',
                    VariableDefinition='particulate thallium quantified by ICP MS', SpeciationCV= 'Tl', NoDataValue=-6999)
print(variable)
print(variable.VariableCode)
DBSession.add(variable)
DBSession.commit()
print("the ID value for our new variable")
print(variable.VariableID)
variable_id = variable.VariableID

<Variables({'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7efdc9a9bd10>, 'SpeciationCV': 'Tl', 'VariableTypeCV': 'Chemistry', 'VariableNameCV': 'Thalium, particulate', 'VariableDefinition': 'particulate thallium quantified by ICP MS', 'VariableCode': 'Tl-particulate', 'NoDataValue': -6999})>
Tl-particulate
the ID value for our new variable
108


<p>8) To finish things off let's retrieve the new variable from the database, display it, and delete it so this script can be run again. </p>

In [11]:
retreived_variable = DBSession.query(Variables).get(variable_id)
print(retreived_variable)
DBSession.delete(retreived_variable)
DBSession.commit()

<Variables({'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7efdc9a9bd10>, 'VariableID': 108, 'SpeciationCV': u'Tl', 'VariableNameCV': u'Thalium, particulate', 'VariableTypeCV': u'Chemistry', 'NoDataValue': Decimal('-6999.0000000000'), 'VariableDefinition': u'particulate thallium quantified by ICP MS', 'VariableCode': u'Tl-particulate'})>
